<a href="https://colab.research.google.com/github/Androesc/Corte2IA/blob/main/IAPrueba.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ejercicio 1 al 4

In [26]:
!pip install bitsandbytes accelerate transformers
!pip install gradio peft transformers accelerate bitsandbytes
!pip install transformers torch
!pip install gradio transformers torch

In [19]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import os

# ✅ Configurar carpeta para caché de modelos en Colab (opcional)
os.environ["TRANSFORMERS_CACHE"] = "/content/model_cache"

def verificar_dispositivo():
    """
    Verifica el dispositivo disponible (CPU/GPU) y muestra información relevante.
    """
    if torch.cuda.is_available():
        dispositivo = torch.device("cuda")
        print("✅ GPU disponible:", torch.cuda.get_device_name(0))
    else:
        dispositivo = torch.device("cpu")
        print("⚠️ Usando CPU (GPU no disponible)")
    return dispositivo

def cargar_modelo(nombre_modelo):
    """
    Carga un modelo preentrenado y su tokenizador desde Hugging Face.
    """
    print(f"📦 Cargando modelo y tokenizador: {nombre_modelo}")
    tokenizador = AutoTokenizer.from_pretrained(nombre_modelo)
    modelo = AutoModelForCausalLM.from_pretrained(nombre_modelo)
    modelo.eval()  # Modo inferencia
    return modelo, tokenizador

def generar_texto(modelo, tokenizador, prompt, dispositivo, max_length=50):
    """
    Genera texto usando el modelo dado un prompt.
    """
    entradas = tokenizador(prompt, return_tensors="pt").to(dispositivo)
    salida = modelo.generate(**entradas, max_length=max_length, pad_token_id=tokenizador.eos_token_id)
    texto_generado = tokenizador.decode(salida[0], skip_special_tokens=True)
    return texto_generado

def main():
    dispositivo = verificar_dispositivo()
    modelo, tokenizador = cargar_modelo("gpt2")
    modelo.to(dispositivo)

    prompt = "Once upon a time"
    print("\n🧠 Prompt de prueba:", prompt)
    resultado = generar_texto(modelo, tokenizador, prompt, dispositivo)
    print("\n📝 Resultado generado:\n", resultado)

if __name__ == "__main__":
    main()


✅ GPU disponible: Tesla T4
📦 Cargando modelo y tokenizador: gpt2

🧠 Prompt de prueba: Once upon a time

📝 Resultado generado:
 Once upon a time, the world was a place of great beauty and great danger. The world was a place of great danger, and the world was a place of great danger. The world was a place of great danger, and the world was a


In [20]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# Verificación del dispositivo
def verificar_dispositivo():
    if torch.cuda.is_available():
        print("✅ GPU disponible:", torch.cuda.get_device_name(0))
        return torch.device("cuda")
    else:
        print("⚠️ Usando CPU (GPU no disponible)")
        return torch.device("cpu")

# Cargar modelo y tokenizador
def cargar_modelo(nombre_modelo):
    tokenizador = AutoTokenizer.from_pretrained(nombre_modelo)
    modelo = AutoModelForCausalLM.from_pretrained(nombre_modelo)
    modelo.eval()
    return modelo, tokenizador

# Preprocesamiento
def preprocesar_entrada(texto, tokenizador, dispositivo, longitud_maxima=512):
    tokens = tokenizador(texto, return_tensors="pt", truncation=True, max_length=longitud_maxima)
    return tokens.to(dispositivo)

# Generación
def generar_respuesta(modelo, entrada_procesada, tokenizador, parametros_generacion=None):
    if parametros_generacion is None:
        parametros_generacion = {
            "max_length": entrada_procesada["input_ids"].shape[1] + 50,
            "temperature": 0.7,
            "top_p": 0.9,
            "do_sample": True,
            "pad_token_id": tokenizador.eos_token_id
        }
    salida = modelo.generate(**entrada_procesada, **parametros_generacion)
    respuesta = tokenizador.decode(salida[0], skip_special_tokens=True)
    return respuesta

# Prompt de sistema
def crear_prompt_sistema(instrucciones):
    return f"Instrucciones del sistema: {instrucciones}\nUsuario: "

# Ejemplo de uso
def interaccion_simple():
    dispositivo = verificar_dispositivo()
    modelo, tokenizador = cargar_modelo("gpt2")
    modelo.to(dispositivo)

    prompt = crear_prompt_sistema("Responde como si fueras un guía turístico amable y divertido.")
    entrada_usuario = "¿Qué me recomiendas visitar en Cali?"
    entrada_completa = prompt + entrada_usuario

    entrada_proc = preprocesar_entrada(entrada_completa, tokenizador, dispositivo)
    respuesta = generar_respuesta(modelo, entrada_proc, tokenizador)

    print("🤖 Respuesta del chatbot:\n", respuesta)

# Ejecutar
interaccion_simple()

✅ GPU disponible: Tesla T4
🤖 Respuesta del chatbot:
 Instrucciones del sistema: Responde como si fueras un guía turístico amable y divertido.
Usuario: ¿Qué me recomiendas visitar en Cali?
Usuario: ¿Ou dell'armez porque una elle en elle mejor que sui año?
Usuario: ¿Qué una elle mejor en


In [21]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# --- Utilidades del Ejercicio 1 y 2 ---
def verificar_dispositivo():
    if torch.cuda.is_available():
        print("✅ GPU disponible:", torch.cuda.get_device_name(0))
        return torch.device("cuda")
    else:
        print("⚠️ Usando CPU (GPU no disponible)")
        return torch.device("cpu")

def cargar_modelo(nombre_modelo):
    tokenizador = AutoTokenizer.from_pretrained(nombre_modelo)
    modelo = AutoModelForCausalLM.from_pretrained(nombre_modelo)
    modelo.eval()
    return modelo, tokenizador

def preprocesar_entrada(texto, tokenizador, dispositivo, longitud_maxima=1024):
    entrada = tokenizador(texto, return_tensors="pt", truncation=True, max_length=longitud_maxima)
    return entrada.to(dispositivo)

def generar_respuesta(modelo, entrada_procesada, tokenizador, parametros_generacion=None):
    if parametros_generacion is None:
        parametros_generacion = {
            "max_length": entrada_procesada["input_ids"].shape[1] + 50,
            "temperature": 0.7,
            "top_p": 0.9,
            "do_sample": True,
            "pad_token_id": tokenizador.eos_token_id
        }
    salida = modelo.generate(**entrada_procesada, **parametros_generacion)
    return tokenizador.decode(salida[0], skip_special_tokens=True)

# --- Clase para gestionar el historial de la conversación ---
class GestorContexto:
    def __init__(self, longitud_maxima=1024, formato_mensaje=None):
        self.historial = []
        self.longitud_maxima = longitud_maxima
        self.formato_mensaje = formato_mensaje or self._formato_predeterminado

    def _formato_predeterminado(self, rol, contenido):
        return f"{rol.capitalize()}: {contenido}"

    def agregar_mensaje(self, rol, contenido):
        self.historial.append((rol, contenido))

    def construir_prompt_completo(self):
        return "\n".join([self.formato_mensaje(rol, contenido) for rol, contenido in self.historial])

    def truncar_historial(self, tokenizador):
        while True:
            prompt = self.construir_prompt_completo()
            tokens = tokenizador(prompt, return_tensors="pt", truncation=False)
            if tokens["input_ids"].shape[1] <= self.longitud_maxima:
                break
            if len(self.historial) > 1:
                self.historial.pop(1)  # Eliminar el segundo mensaje más antiguo (conserva el sistema)
            else:
                break

# --- Clase principal del chatbot con contexto ---
class Chatbot:
    def __init__(self, modelo_id, instrucciones_sistema=None):
        self.modelo, self.tokenizador = cargar_modelo(modelo_id)
        self.dispositivo = verificar_dispositivo()
        self.modelo.to(self.dispositivo)
        self.gestor_contexto = GestorContexto()

        if instrucciones_sistema:
            self.gestor_contexto.agregar_mensaje("sistema", instrucciones_sistema)

    def responder(self, mensaje_usuario, parametros_generacion=None):
        self.gestor_contexto.agregar_mensaje("usuario", mensaje_usuario)
        self.gestor_contexto.truncar_historial(self.tokenizador)

        prompt = self.gestor_contexto.construir_prompt_completo()
        entrada = preprocesar_entrada(prompt, self.tokenizador, self.dispositivo)
        respuesta = generar_respuesta(self.modelo, entrada, self.tokenizador, parametros_generacion)

        # Extraer solo la parte nueva generada (evitar repetir todo el prompt)
        respuesta_extraida = respuesta[len(prompt):].strip().split("\n")[0]
        self.gestor_contexto.agregar_mensaje("asistente", respuesta_extraida)
        return respuesta_extraida

# --- Simulación de conversación de prueba ---
def prueba_conversacion():
    bot = Chatbot("gpt2", instrucciones_sistema="Eres un asistente amigable de turismo.")

    print("Usuario: ¿Qué lugares turísticos hay en Cali?")
    print("Asistente:", bot.responder("¿Qué lugares turísticos hay en Cali?"))

    print("\nUsuario: ¿Y qué me recomiendas para comer?")
    print("Asistente:", bot.responder("¿Y qué me recomiendas para comer?"))

    print("\nUsuario: ¿Hay algo divertido por la noche?")
    print("Asistente:", bot.responder("¿Hay algo divertido por la noche?"))

# Ejecutar
prueba_conversacion()


✅ GPU disponible: Tesla T4
Usuario: ¿Qué lugares turísticos hay en Cali?
Asistente: Usuario: La más años de Cali!

Usuario: ¿Y qué me recomiendas para comer?
Asistente: Asistente: Usuario: ¿Puede, tu vuelo para de cali?

Usuario: ¿Hay algo divertido por la noche?
Asistente: Asistente: Asistente: Usuario: ¿Usuario?


In [22]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch
import time
import gc

# Cuantización: usar bitsandbytes para 4 u 8 bits
def configurar_cuantizacion(bits=4):
    if bits not in [4, 8]:
        raise ValueError("Solo se permiten 4 u 8 bits.")
    config_cuantizacion = BitsAndBytesConfig(
        load_in_4bit=(bits == 4),
        load_in_8bit=(bits == 8),
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4"
    )
    return config_cuantizacion

# Cargar modelo con cuantización y otras optimizaciones (solo en GPUs compatibles)
def cargar_modelo_optimizado(nombre_modelo, optimizaciones=None):
    from transformers import AutoModelForCausalLM, AutoTokenizer

    if optimizaciones is None:
        optimizaciones = {
            "cuantizacion": True,
            "bits": 4,
            "offload_cpu": False,
            "flash_attention": False  # solo si compatible
        }

    tokenizer = AutoTokenizer.from_pretrained(nombre_modelo)

    if optimizaciones["cuantizacion"]:
        config = configurar_cuantizacion(bits=optimizaciones["bits"])
        modelo = AutoModelForCausalLM.from_pretrained(
            nombre_modelo,
            quantization_config=config,
            device_map="auto"
        )
    else:
        modelo = AutoModelForCausalLM.from_pretrained(nombre_modelo).to("cuda" if torch.cuda.is_available() else "cpu")

    return modelo, tokenizer

# (Opcional) Sliding window: usado en transformers para atención eficiente (requiere modelo compatible)
def aplicar_sliding_window(modelo, window_size=1024):
    if hasattr(modelo.config, "max_position_embeddings"):
        modelo.config.sliding_window = window_size
        print(f"✅ Ventana deslizante activada: {window_size} tokens")
    else:
        print("⚠️ Este modelo no soporta sliding window nativamente.")

# Evaluar rendimiento: tiempo, memoria y tokens/s
def evaluar_rendimiento(modelo, tokenizador, texto_prueba, dispositivo):
    torch.cuda.empty_cache()
    gc.collect()

    entrada = tokenizador(texto_prueba, return_tensors="pt").to(dispositivo)
    t0 = time.time()
    with torch.no_grad():
        salida = modelo.generate(**entrada, max_length=entrada["input_ids"].shape[1] + 50)
    t1 = time.time()

    tiempo_total = t1 - t0
    tokens_generados = salida.shape[1]
    tokens_por_segundo = tokens_generados / tiempo_total

    memoria = torch.cuda.memory_allocated(dispositivo) / (1024 ** 2) if dispositivo.type == "cuda" else 0
    return {
        "tiempo_inferencia_seg": round(tiempo_total, 3),
        "tokens_generados": tokens_generados,
        "tokens_por_segundo": round(tokens_por_segundo, 2),
        "memoria_MB": round(memoria, 2)
    }

# Demostración comparativa
def demo_optimizaciones():
    texto = "Explícame en pocas palabras qué es un modelo de lenguaje de gran tamaño."
    dispositivo = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    configuraciones = {
        "base_sin_opt": {"cuantizacion": False},
        "cuantizado_4bit": {"cuantizacion": True, "bits": 4},
        # "con_sliding": {"cuantizacion": True, "bits": 4, "flash_attention": False},  # si el modelo soporta
    }

    resultados = {}

    for nombre_conf, conf in configuraciones.items():
        print(f"\n🔧 Evaluando: {nombre_conf}")
        modelo, tokenizer = cargar_modelo_optimizado("tiiuae/falcon-rw-1b", optimizaciones=conf)
        modelo.to(dispositivo)
        resultado = evaluar_rendimiento(modelo, tokenizer, texto, dispositivo)
        resultados[nombre_conf] = resultado
        print("📊", resultado)
        del modelo
        gc.collect()
        torch.cuda.empty_cache()

    return resultados

# Ejecutar la demo
demo_optimizaciones()



🔧 Evaluando: base_sin_opt


tokenizer_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

...

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


📊 {'tiempo_inferencia_seg': 1.907, 'tokens_generados': 78, 'tokens_por_segundo': 40.89, 'memoria_MB': 6494.74}

🔧 Evaluando: cuantizado_4bit


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


📊 {'tiempo_inferencia_seg': 3.074, 'tokens_generados': 78, 'tokens_por_segundo': 25.38, 'memoria_MB': 2282.39}


{'base_sin_opt': {'tiempo_inferencia_seg': 1.907,
  'tokens_generados': 78,
  'tokens_por_segundo': 40.89,
  'memoria_MB': 6494.74},
 'cuantizado_4bit': {'tiempo_inferencia_seg': 3.074,
  'tokens_generados': 78,
  'tokens_por_segundo': 25.38,
  'memoria_MB': 2282.39}}

# Ejercicio Final

In [30]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import gradio as gr

# Clase para manejar el contexto de la conversación
class GestorContexto:
    def __init__(self, longitud_maxima=1024, formato_mensaje=None):
        self.historial = []
        self.longitud_maxima = longitud_maxima
        self.formato_mensaje = formato_mensaje or self._formato_predeterminado

    def _formato_predeterminado(self, rol, contenido):
        return f"{rol}: {contenido}"

    def agregar_mensaje(self, rol, contenido):
        mensaje_formateado = self.formato_mensaje(rol, contenido)
        self.historial.append(mensaje_formateado)

    def construir_prompt_completo(self):
        # Solo mantenemos los últimos mensajes que sean relevantes
        return "\n".join(self.historial[-5:])  # Tomamos solo los últimos 5 mensajes

    def truncar_historial(self, tokenizador):
        while True:
            tokens = tokenizador.encode(self.construir_prompt_completo(), return_tensors="pt")
            if len(tokens[0]) > self.longitud_maxima:
                self.historial.pop(0)
            else:
                break

# Función para cargar el modelo optimizado (cuantización opcional)
def cargar_modelo_optimizado(nombre_modelo):
    modelo = AutoModelForCausalLM.from_pretrained(nombre_modelo)
    tokenizador = AutoTokenizer.from_pretrained(nombre_modelo)

    # Verificar si el tokenizador tiene un pad_token, si no, usar el eos_token como pad_token
    if tokenizador.pad_token is None:
        tokenizador.pad_token = tokenizador.eos_token  # Usamos el token EOS como PAD

    dispositivo = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    modelo.to(dispositivo)
    return modelo, tokenizador, dispositivo

# Preprocesamiento de la entrada
def preprocesar_entrada(texto, tokenizador, longitud_maxima=512, dispositivo=None):
    tokens = tokenizador.encode(texto, truncation=True, padding="max_length", max_length=longitud_maxima, return_tensors="pt")
    tokens = tokens.to(dispositivo)
    return tokens

# Generación de la respuesta
def generar_respuesta(modelo, entrada_procesada, tokenizador, parametros_generacion=None):
    if parametros_generacion is None:
        parametros_generacion = {
            "do_sample": True,
            "max_new_tokens": 50,
            "temperature": 0.7,
            "top_p": 0.9,
            "top_k": 50,
        }

    with torch.no_grad():
        salida = modelo.generate(entrada_procesada, **parametros_generacion)

    respuesta = tokenizador.decode(salida[0], skip_special_tokens=True)
    return respuesta

# Clase del chatbot
class Chatbot:
    def __init__(self, modelo_id, instrucciones_sistema=None):
        self.modelo, self.tokenizador, self.dispositivo = cargar_modelo_optimizado(modelo_id)
        self.gestor_contexto = GestorContexto()

        if instrucciones_sistema:
            self.gestor_contexto.agregar_mensaje('sistema', instrucciones_sistema)

    def responder(self, mensaje_usuario):
        self.gestor_contexto.agregar_mensaje('usuario', mensaje_usuario)
        prompt_completo = self.gestor_contexto.construir_prompt_completo()
        entrada_procesada = preprocesar_entrada(prompt_completo, self.tokenizador, dispositivo=self.dispositivo)
        respuesta = generar_respuesta(self.modelo, entrada_procesada, self.tokenizador)
        # Agregar la respuesta al historial
        self.gestor_contexto.agregar_mensaje('asistente', respuesta)
        # Retornar solo la respuesta generada (sin los mensajes anteriores)
        return respuesta.strip()

# Función para crear la interfaz web con Gradio
def crear_interfaz_web(chatbot):
    """
    Crea una interfaz web simple para el chatbot usando Gradio.

    Args:
        chatbot: Instancia del chatbot

    Returns:
        gr.Interface: Interfaz de Gradio
    """
    # Función de callback para procesar entradas y generar respuestas
    def procesar_entrada(mensaje_usuario):
        return chatbot.responder(mensaje_usuario)

    # Definir los componentes de la interfaz
    interfaz = gr.Interface(
        fn=procesar_entrada,  # Función de procesamiento
        inputs=gr.Textbox(lines=2, placeholder="Escribe tu mensaje aquí..."),  # Entrada de texto
        outputs=gr.Textbox(),  # Salida de texto
        title="Chatbot Personalizado",  # Título de la interfaz
        description="Chatbot que responde preguntas y mantiene el contexto de la conversación."  # Descripción
    )

    return interfaz

# Función principal para el despliegue
def main_despliegue():
    # Cargar el modelo personalizado
    modelo_id = "gpt2"  # O usar tu modelo personalizado si lo tienes
    instrucciones_sistema = "El chatbot debe ser amigable, educado y conciso."

    # Crear instancia del chatbot
    chatbot = Chatbot(modelo_id, instrucciones_sistema)

    # Crear y lanzar la interfaz web
    interfaz = crear_interfaz_web(chatbot)
    interfaz.launch(share=True)  # share=True para obtener un enlace público de la interfaz

if __name__ == "__main__":
    main_despliegue()


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://62b0fec2b0b0d05261.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
